In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#数据加载
train_data = datasets.CIFAR10(root='.', train=True, transform=transforms.ToTensor(), download = False)
test_data = datasets.CIFAR10(root='.', train = False, transform=transforms.ToTensor(), download=False)

In [2]:
#超参数定义
EPOCH = 10
BATCH_SIZE = 30
LR = 0.001


In [ ]:
'''
####################执行本模块会有kernel restart的错误########################
#图像展示
temp = train_data[1][0].numpy()
#print(temp.shape) #(3, 32, 32) need to be transpose as (32, 32, 3)
temp = temp.transpose(1, 2, 0)
print(temp.shape)
plt.imshow(temp)
plt.show()
###########################################################################

Problem:
The problem is that the image box is using the same Python process as the kernel. When you execute this from an interactive shell, the shell waits for you to get back from the image display box. But Jupyter wants to claim control back after you executed a command block. Therefore it is not happy to wait for the window, and you lose control of the Python process, crashing everything.
Solution:
To solve this, you need to explicitly close the window when you are done with it. To accomplish this, you can use the cv2.destroyAllWindows() functionality.
Since you probably don’t want your screen to close immediately, you can tell OpenCV to wait for a keypress. You can specify which key, but it is best to accept any key
Tips:
To exit the window, DON’T CLICK THE RED CROSS, just tap any key on your keyboard!If you do close with the cross, you won’t let OpenCV close the box for you, and Jupyter will stay waiting, causing you to have to reset the kernel yourself.
'''

#解决方案如下
import cv2
temp = train_data[1][0].numpy()
#print(temp.shape) 
temp = temp.transpose(1, 2, 0)
cv2.imshow('test temp',temp) 
cv2.waitKey(0)  #wait for any keyboard input to close the image window
cv2.destroyAllWindows() #close the image window

(32, 32, 3)


In [4]:
#DataLoader分批加载 batch
train_load = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True )
test_load = DataLoader(dataset = test_data, batch_size = BATCH_SIZE, shuffle = True )


In [5]:
#ResNet50 
model = torchvision.models.resnet50(pretrained = False)

#Loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

#use GPU 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
## model and inputdata - to(device)
model = model.to(device)


In [6]:
#训练过程
for epoch in range(EPOCH):
    for i, data in enumerate(train_load):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #前向传播
        outputs = model(inputs)
        #计算Loss
        loss = criterion(outputs, labels)
        #梯度清零
        optimizer.zero_grad()
        #反向传播
        loss.backward()
        #参数更新
        optimizer.step()
    print('epoch{}, loss{:.4f}'.format(epoch+1, loss.item()))
    
torch.save(model, 'cifar10_resnet.pt')
print('cifar10_resnet.pt saved')

epoch1, loss1.8213
epoch2, loss1.1256
epoch3, loss1.4377
epoch4, loss1.9522
epoch5, loss0.9694
epoch6, loss1.0403
epoch7, loss1.0708
epoch8, loss0.8111
epoch9, loss0.6763
epoch10, loss0.3536
cifar10_resnet.pt saved


In [7]:
#模型加载
model = torch.load('cifar10_resnet.pt')
#测试
model.eval() #指定model在BN和Dropout等操作时使用训练好的参数
correct, total = 0, 0
for data in test_load:
    inputs, labels = inputs.to(device), labels.to(device)
    #front propagation
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total = total + labels.size(0)
    correct = correct + (predicted == labels).sum().item()

#输出测试集准确率
print('test accuracy: {:.4f}%'.format(100.0*correct/total))

test accuracy: 90.0000%
